# 猫狗大战 毕业项目——Inception_V3 迁移

## 开始
导入一切并我们设置所使用的GPU。
- dev1: Tesla K40c

In [1]:
%matplotlib inline
#import utilities
import os
import shutil
import numpy as np
import random
from tqdm import tqdm  
from time import time
from PIL import Image
import h5py
import pandas as pd
from helper import *

from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.callbacks import *
from keras.optimizers import *
from keras.utils import *
from keras import backend as K

#如果系统上有多块GPU，“0”可以替换成其它GPU的编号
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

/home/pengjun/.conda/envs/keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 数据文件处理
训练数据包括12500张猫的图片和12500张狗的图片。我们为数据文件建立symbol link并划分为训练集和验证集，所使用的方法参考了[这里](https://github.com/ypwhs/dogs_vs_cats)。

In [2]:
#为数据连理symbol-link
train_data_dir, valid_data_dir, test_data_dir = prepare_data_file()

100%|██████████| 12500/12500 [00:00<00:00, 191037.12it/s]


In [3]:
#导入训练数据和测试数据
X_train, Y_train, X_test = load_feature_data("feature_inception_v3.h5")

In [5]:
#构造模型并显示所有网络层的名称
input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [6]:
#训练模型并导出权重参数
filepath="inception-tune0-best_weight.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_weights_only=True)
callbacks_list = [checkpoint]
model.fit(X_train, Y_train, batch_size=128, epochs=20, validation_split=0.2, shuffle=True,
         callbacks=callbacks_list)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 1s 45us/step - loss: 0.1293 - acc: 0.9665 - val_loss: 0.0404 - val_acc: 0.9906

Epoch 00001: val_loss improved from inf to 0.04039, saving model to inception-tune0-best_weight.h5
Epoch 2/20
20000/20000 [==============================] - 0s 22us/step - loss: 0.0405 - acc: 0.9896 - val_loss: 0.0243 - val_acc: 0.9926

Epoch 00002: val_loss improved from 0.04039 to 0.02434, saving model to inception-tune0-best_weight.h5
Epoch 3/20
20000/20000 [==============================] - 0s 22us/step - loss: 0.0314 - acc: 0.9911 - val_loss: 0.0211 - val_acc: 0.9932

Epoch 00003: val_loss improved from 0.02434 to 0.02110, saving model to inception-tune0-best_weight.h5
Epoch 4/20
20000/20000 [==============================] - 0s 24us/step - loss: 0.0290 - acc: 0.9919 - val_loss: 0.0191 - val_acc: 0.9934

Epoch 00004: val_loss improved from 0.02110 to 0.01906, saving model to inception-tune0-best_w

In [7]:
#在测试集上进行预测并导出预测值
predict_on_model(test_data_dir, X_test, model, "pred-inception-tune0.csv")

12500/12500 [==============================] - 0s 38us/step
Found 12500 images belonging to 1 classes.


/home/pengjun/DLND/Cat_vs_Dog/helper.py:130: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(index-1, 'label', y_test[i])


> 这一模型在Kaggle上的得分为0.04117.